In [3]:
import pandas as pd
import numpy as np
import sklearn
import sys
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
import xgboost as xgb

def write_hyperparams(hyperparams, fileName):
    f = open(fileName, 'w')
    f.write(' '.join(str(x) for x in hyperparams) + '\n')
    f.close()

def read_hyperparams(fileName):
    f = open(fileName, 'r')
    n_folds, n_estimators, max_depth, max_features, \
        score = f.read().split()
    return int(n_folds), int(n_estimators), int(max_depth), \
        max_features, float(score)

# define some constants
xgboost_data_dir = '../xgboost_data/'
data_dir = '../data/'
hyperParamFile = 'hyperparams_cat.txt'
n_folds = 3
verbose_tree = 3
verbose_grid = 0
n_jobs = 3
random_state = 1
criterion = 'mae'

# For xgboost
param = {}
param['objective'] = 'reg:gamma'
param['nthread'] = 2
param['eval_metric'] = 'mae'
param['max_depth'] = 4
param['eta'] = 0.2
param['silent'] = 1  # prints messages to screen (1 silences these)
param['tree_method'] = 'auto'
param['lambda'] = 0.1
num_boost_rounds = 100

# allows for validation to occur during training
evallist = []

# read in the train dataset
print('loading training data...')
# for xgboost
dtrain = xgb.DMatrix(xgboost_data_dir+'continuous_selected.dat')

evaluations = xgb.cv(params=param, dtrain=dtrain, num_boost_round=num_boost_rounds,
            nfold=10)
print(evaluations)


tree = xgb.train( param, dtrain, num_boost_rounds, evallist)
tree.save_model('prac.model')

dtest = xgb.DMatrix(xgboost_data_dir+'continuous_test_selected.dat')
y_pred = tree.predict(dtest)

print(y_pred)


loading training data...
    test-mae-mean  test-mae-std  train-mae-mean  train-mae-std
0     3036.694434     24.421653     3036.694434       2.713561
1     3036.559253     24.421653     3036.559228       2.713542
2     3036.394189     24.421687     3036.394189       2.713542
3     3036.192505     24.421681     3036.192529       2.713542
4     3035.946387     24.421653     3035.946313       2.713488
5     3035.645679     24.421653     3035.645630       2.713474
6     3035.278418     24.421688     3035.278467       2.713542
7     3034.830054     24.421671     3034.830029       2.713535
8     3034.282568     24.421641     3034.282568       2.713511
9     3033.614087     24.421684     3033.614136       2.713521
10    3032.798096     24.421684     3032.797998       2.713542
11    3031.801880     24.421714     3031.801880       2.713521
12    3030.585889     24.421738     3030.585864       2.713502
13    3029.102002     24.421735     3029.101929       2.713532
14    3027.291358     24.42171

In [ ]:
import pandas as pd
import numpy as np
import sklearn
import sys
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
import xgboost as xgb

def write_hyperparams(hyperparams, fileName):
	f = open(fileName, 'w')
	f.write(' '.join(str(x) for x in hyperparams) + '\n')
	f.close()

def read_hyperparams(fileName):
	f = open(fileName, 'r')
	n_folds, n_estimators, max_depth, max_features, \
		score = f.read().split()
	return int(n_folds), int(n_estimators), int(max_depth), \
		max_features, float(score)

# define some constants
hyperParamFile = 'hyperparams_cont.txt'
n_folds = 3
verbose_tree = 3
verbose_grid = 0
n_jobs = 4
random_state = 1
criterion = 'mae'

# read in the train dataset
print('loading training data...')
df_cont = pd.read_csv('continuous_selected.csv', header=0, index_col=0)
df_y = pd.read_csv('y_all.csv', header=0, index_col=0)

X = df_cont.values
y = np.ravel(df_y.values)

# create a Random Forest Classifier
print('creating a model...')
n_estimators_def = 100
max_features_def = 'sqrt'
max_depth_def = 10
#n_estimators_range = [10, 50]
#max_features_range = ['sqrt']
#max_depth_range = [10, 30]
#param_grid = {'n_estimators': n_estimators_range, 'max_features': max_features_range,
#	'max_depth': max_depth_range}

# create a tree to train the models on
tree = RandomForestRegressor(criterion=criterion, verbose=verbose_tree, n_jobs=n_jobs,
	random_state=random_state, n_estimators=n_estimators_def,
	max_features=max_features_def, max_depth=max_depth_def)

# some feature selection
#print('selecting features...')
#print('input feature shape: ')
#print(X.shape)
#tree.fit(X, y)
#feature_select = SelectFromModel(tree, prefit=True, threshold=feature_threshold)
##print(tree.feature_importances_.sort())
#X_new = feature_select.transform(X)
#print('new input feature shape: ')
#print(X_new.shape)

# perform a grid search to tune the paramteres
#print('grid search to tune hyperparameters...')
#gs = GridSearchCV(estimator=tree,
#	param_grid=param_grid, scoring=None,
#	cv=n_folds, n_jobs=n_jobs, verbose=verbose_grid)
#gs = gs.fit(X_new, y)
#print(gs.scorer_)
#print('best score from grid search: %.3f' % gs.best_score_)
#print(gs.best_params_)
#best = gs.best_params_
#n_estimators_gs = best['n_estimators']
#max_depth_gs = best['max_depth']
#max_features_gs = best['max_features']

# run some cross validation
print('running cross validation to determine accuracy of model...')
scores = []
splits = KFold(n_splits=n_folds, shuffle=True, random_state=random_state)
for train, test in splits.split(X):
	tree.fit(X[train], y[train])
	predicted = tree.predict(X[test])
	score = mean_absolute_error(y[test], predicted)
	scores.append(score)
print(scores)

# determine which features to write to the file
n_estimators = n_estimators_def
max_depth = max_depth_def
max_features=max_features_def
score = np.mean(scores)

print('writing the data to file...')
params = (n_folds, n_estimators, max_depth, max_features, score)
write_hyperparams(params, hyperParamFile)
n_folds, n_estimators, max_depth, max_features, \
	score = read_hyperparams(hyperParamFile)
print(n_folds, n_estimators, max_depth, max_features, score)